In [155]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama
import pandas as pd
import json

## 提取出文本中的物品

In [156]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOllama(temperature=0, model="qwen:32b-chat-v1.5-q5_K_M")

text_input = """
林克推开门进入卧室，房间布置简洁而温馨。一张坚固的木床靠在北墙正中，床头上方挂着一幅描绘着塞尔达大陆的装饰画，床尾朝向南边的窗户，让林克在清晨醒来时能第一时间感受到阳光的温暖。
床的右侧是一个小巧的书桌，书桌面向东边，林克可以在这里规划他的冒险路线，书桌上整齐地摆放着地图和羽毛笔。在房间的西南角，一个高挑的衣柜紧贴墙壁，柜门上刻有海拉尔的纹章，内部整齐地挂着林克的服装和装备。
衣柜对面，一张舒适的靠背椅放置在东墙边，旁边是一张小茶几，上面常放着一杯热茶，供林克在休息时享用。整个房间的布局以方便和实用为主，同时不失为一个放松和规划冒险的理想空间。
"""

get_object_chain = ChatPromptTemplate.from_template("请提取出如下文本中的陈设和物品，输出一个数组。文本：{text_input}") | chat_model | StrOutputParser()

objects_in_text = get_object_chain.invoke({"text_input": text_input})

print(objects_in_text)

['木床', '装饰画', '书桌', '地图', '羽毛笔', '衣柜', '海拉尔的纹章', '服装', '装备', '靠背椅', '小茶几', '热茶']


## 在素材库中找到匹配的物品

In [195]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_in_gallery", description="这是你觉得与用户输入最匹配的物品"),
    ResponseSchema(name="objects_url_in_gallery", description="这是这个最匹配的物品的URL"),
    ResponseSchema(name="match_score", description="你认为用户输入的匹配与你匹配的接近程度的0-100分")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。
已入库物品的URL和已入库物品是一一对应关系。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

已入库物品:
{objects_in_gallery}

已入库物品URL:
{objects_url_in_gallery}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "objects_in_gallery"],
    partial_variables={"format_instructions": format_instructions}
)

# Get your standardized names. You can swap this out with whatever list you want!
df = pd.read_csv('../data/ObjectsInGallery.csv')
objects_in_gallery = ", ".join(df['物品'].values)
objects_url_in_gallery = ", ".join(df['URL'].values)

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, objects_in_gallery=objects_in_gallery, objects_url_in_gallery=objects_url_in_gallery)

print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

# 提取JSON内容
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsMatched.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"objects_url_in_gallery": string  // 这是这个最匹配的物品的URL
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。
已入库物品的URL和已入库物品是一一对应关系。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"objects_url_in_gallery": string  // 这是这个最匹配的物品的URL
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
['木床', '装饰画', '书桌', '地图', '羽毛笔', '衣柜', '海拉尔的纹章', '服装', '

## 根据文本确定物品方位和坐标

In [196]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_name", description="这是用户输入的物品的英语翻译（合法的C语言变量名称形式）"),
    ResponseSchema(name="place", description="这是根据上下文物品的位置"),
    ResponseSchema(name="direction", description="这是根据上下文物品的朝向（东、西、南、北、上、下）"),
    ResponseSchema(name="translation_x", description="这是物品的位置X坐标"),
    ResponseSchema(name="translation_y", description="这是物品的位置Y坐标"),
    ResponseSchema(name="translation_z", description="这是物品的位置Z坐标"),
    ResponseSchema(name="rotation_x", description="这是物品的X旋转角"),
    ResponseSchema(name="rotation_y", description="这是物品的Y旋转角"),
    ResponseSchema(name="rotation_z", description="这是物品的Z旋转角")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称。
根据上下文推断出物品的位置和朝向。
重点考虑物品之间的相对位置关系，位置应符合上下文的描述，同时避免重叠和碰撞。
如果有描述边界，物品不要超出边界。
X轴指向东、Y轴指向北、Z轴指向上。
物体旋转角为欧拉旋转角

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

上下文:
{text_input}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "text_input"],
    partial_variables={"format_instructions": format_instructions}
)

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, text_input=text_input)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsLayout.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式）
	"place": string  // 这是根据上下文物品的位置
	"direction": string  // 这是根据上下文物品的朝向（东、西、南、北、上、下）
	"translation_x": string  // 这是物品的位置X坐标
	"translation_y": string  // 这是物品的位置Y坐标
	"translation_z": string  // 这是物品的位置Z坐标
	"rotation_x": string  // 这是物品的X旋转角
	"rotation_y": string  // 这是物品的Y旋转角
	"rotation_z": string  // 这是物品的Z旋转角
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
根据上下文推断出物品的位置和朝向。
重点考虑物品之间的相对位置关系，位置应符合上下文的描述，同时避免重叠和碰撞。
如果有描述边界，物品不要超出边界。
X轴指向东、Y轴指向北、Z轴指向上。
物体旋转角为欧拉旋转角

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_name":

## 将JSON文件转换为USDA文件

In [210]:
import json
import os
from pxr import Usd, UsdGeom, Sdf, Gf

# 定义一个函数来创建USD对象
def create_usd_object(stage, prim_path, obj_name, obj_url, position, rotation):
    # 创建一个新的Prim在给定的路径上
    prim = stage.DefinePrim(prim_path, 'Xform')
    prim.name = obj_name

    xformable = UsdGeom.Xformable(prim)

    xformable.AddTranslateOp().Set(value=translation)
    xformable.AddRotateXYZOp().Set(value=rotation)

    # 创建payload，并设置其references指向目标USD文件
    payload = Sdf.Payload(obj_url)
    prim.GetPayloads().AddPayload(payload)

    print(xformable)
    
    return prim


# 创建USD stage
stage = Usd.Stage.CreateNew('../data/link_room_layout_25.usda')

# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

with open('../data/ObjectsLayout.json', 'r') as f:
    ObjectsLayout = json.load(f)

# # 合并两个JSON文件的数据
for obj in ObjectsMatched:
    if int(obj['match_score']) <= 0:
        continue

    for obj_layout in ObjectsLayout:
        if obj_layout['object_in_text'] == obj['object_in_text']:
            obj_layout.update(obj)

for o in ObjectsLayout:
    print(o)

for o in ObjectsMatched:
    print(o)

# 遍历JSON数据，为每个对象创建一个USD prim
for obj in ObjectsLayout:
    if not 'objects_url_in_gallery' in obj:
        continue

    prim_path = '/Objects/' + obj['object_name']
    translation = Gf.Vec3f(float(obj['translation_x']), float(obj['translation_y']), float(obj['translation_z']))
    rotation = Gf.Vec3f(float(obj['rotation_x']), float(obj['rotation_y']), float(obj['rotation_z']))
    create_usd_object(stage, prim_path, obj['object_in_text'], obj['objects_url_in_gallery'], translation, rotation)

# 保存USD stage
stage.Save()


{'object_in_text': '木床', 'object_name': 'wooden_bed', 'place': '北墙正中', 'direction': '无', 'translation_x': '0', 'translation_y': '-1', 'translation_z': '0', 'rotation_x': '0', 'rotation_y': '0', 'rotation_z': '0', 'object_in_gallery': '床1', 'objects_url_in_gallery': '/home/huangwei1/Downloads/Furniture/Bed_01/Bed_01.usd', 'match_score': '95'}
{'object_in_text': '装饰画', 'object_name': 'ornamental_painting', 'place': '床头上方', 'direction': '无', 'translation_x': '0', 'translation_y': '-1.5', 'translation_z': '0', 'rotation_x': '0', 'rotation_y': '0', 'rotation_z': '0'}
{'object_in_text': '书桌', 'object_name': 'desk', 'place': '床的右侧', 'direction': '东', 'translation_x': '1', 'translation_y': '-1', 'translation_z': '0', 'rotation_x': '0', 'rotation_y': '90', 'rotation_z': '0', 'object_in_gallery': '书桌1', 'objects_url_in_gallery': '/home/huangwei1/Downloads/Furniture/Desk_001/Desk_001.usd', 'match_score': '95'}
{'object_in_text': '地图', 'object_name': 'map', 'place': '书桌上', 'direction': '无', 'trans